## Get system environment info

In [ ]:
import os
env_dist = os.environ
for key in env_dist:
    print(key + ':' + env_dist[key])

## Get system path info

In [ ]:
import sys
sys.path

***
## Lab1: inference sample code using sync API (for iv3/mn/vgg16 model)
### *arg_model_xml* : the model xml (generated from mo.py)
### *arg_model_bin* : the model bin (generated from mo.py)
### *arg_device* : the device to run inferences, can be one of CPU/GPU/MYRIAD
### *arg_image_number* : total infer requests to run

In [ ]:
#!/usr/bin/env python3
# sync api demo
from openvino.inference_engine import IENetwork, IEPlugin
from keras.applications.mobilenet import preprocess_input
import cv2
import os
import time
import sys
import random
import glob

#mobilenet
arg_model_xml="./tf_model/mn/FP16/all_layers.mn.xml"
arg_model_bin="./tf_model/mn/FP16/all_layers.mn.bin"

#vgg16
#arg_model_xml="./tf_model/vgg16/FP16/all_layers.vgg16.xml"
#arg_model_bin="./tf_model/vgg16/FP16/all_layers.vgg16.bin"

#inception V3
#arg_model_xml="./tf_model/iv3/FP16/inception_layers.iv3.xml"
#arg_model_bin="./tf_model/iv3/FP16/inception_layers.iv3.bin"
arg_device="MYRIAD"
arg_image_number = 50

file_list = glob.glob("./test/*/*")

arg_cpu_extension=None

arg_labels="./mn-labels.txt"
label_file = open(arg_labels, "r")
labels = label_file.read().split('\n')

plugin = IEPlugin(device=arg_device)
net = IENetwork(model=arg_model_xml, weights=arg_model_bin)
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
n, c, h, w = net.inputs[input_blob].shape
exec_net = plugin.load(network=net, num_requests=1)

start_time = time.time()
for i in range(1, 1 + arg_image_number):
    img_path = random.choice(file_list)
    img_cat = os.path.split(os.path.dirname(img_path))[1]
    image = cv2.imread(img_path)
    image = cv2.resize(image, (w, h))
    image = image.transpose((2, 0, 1))
    image = image.reshape((n, c, h, w))
    image = preprocess_input(image)
    res = exec_net.infer(inputs={input_blob: image}).get(out_blob)
    top = res[0].argsort()[-1:][::-1]
    pred_label = labels[top[0]]
    print("image category:", '{:25s}'.format(img_cat), "predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])

duration = time.time() - start_time
print("inferred frames: " + str(i) + ", average fps: " + str(i/duration) +"\r", end = '', flush = False)

del exec_net
del net
del plugin

## Lab2: inference sample code using async API (for iv3/mn/vgg16 model)
### *arg_model_xml* : the model xml (generated from mo.py)
### *arg_model_bin* : the model bin (generated from mo.py)
### *arg_device* : the device to run inferences, can be one of CPU/GPU/MYRIAD
### *arg_image_number* : total infer requests to run

In [ ]:
#!/usr/bin/env python3
# async api demo
from openvino.inference_engine import IENetwork, IEPlugin
from keras.applications.mobilenet import preprocess_input
import cv2
import os
import time
import sys
import random
import glob

# args
arg_model_xml="./tf_model/mn/FP16/all_layers.mn.xml"
arg_model_bin="./tf_model/mn/FP16/all_layers.mn.bin"
arg_device="MYRIAD"
arg_image_number = 50

file_list = glob.glob("./test/*/*")

arg_cpu_extension=None

arg_labels="./mn-labels.txt"
label_file = open(arg_labels, "r")
labels = label_file.read().split('\n')

plugin = IEPlugin(device=arg_device)
net = IENetwork(model=arg_model_xml, weights=arg_model_bin)
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
n, c, h, w = net.inputs[input_blob].shape
exec_net = plugin.load(network=net, num_requests=2)

start_time = time.time()
current_inference, next_inference = 0, 1
current_img, next_img = None, None
inferred_images = 0
for i in range(0, arg_image_number):
    img_path = random.choice(file_list)
    img_cat = os.path.split(os.path.dirname(img_path))[1]
    image = cv2.imread(img_path)
    image = cv2.resize(image, (w, h))
    image = image.transpose((2, 0, 1))
    image = image.reshape((n, c, h, w))
    image = preprocess_input(image)
    exec_net.start_async(request_id=current_inference, inputs={input_blob: image})
    current_img = img_cat
    if exec_net.requests[next_inference].wait(-1) == 0:
        inferred_images = inferred_images + 1
        res = exec_net.requests[next_inference].outputs[out_blob]
        top = res[0].argsort()[-1:][::-1]
        pred_label = labels[top[0]]
        print("image category:", '{:25s}'.format(next_img), "predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])
    current_inference, next_inference = next_inference, current_inference
    current_img, next_img = next_img, current_img
# one more inference result left to check
if exec_net.requests[next_inference].wait(-1) == 0:
    inferred_images = inferred_images + 1
    res = exec_net.requests[next_inference].outputs[out_blob]
    top = res[0].argsort()[-1:][::-1]
    pred_label = labels[top[0]]
    print("image category:", '{:25s}'.format(next_img), "predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])

duration = time.time() - start_time
print("inferred frames:", inferred_images ,"average fps:",inferred_images/duration)

del exec_net
del net
del plugin

## Lab3: inference sample code using async API, with multi-thread support  (for iv3/mn/vgg16 model)
### *arg_model_xml* : the model xml (generated from mo.py)
### *arg_model_bin* : the model bin (generated from mo.py)
### *arg_device* : the device to run inferences, can be one of CPU/GPU/MYRIAD
### *arg_image_number* : total infer requests to run

In [ ]:
#!/usr/bin/env python3
# async api + multi-threads
from openvino.inference_engine import IENetwork, IEPlugin
from keras.applications.mobilenet import preprocess_input
import cv2
import os
import time
import sys
import random
import glob

import numpy as np

from multiprocessing import Process, Queue
import multiprocessing
import threading
import queue

# args
arg_model_xml="./tf_model/mn/FP16/all_layers.mn.xml"
arg_model_bin="./tf_model/mn/FP16/all_layers.mn.bin"
arg_device="MYRIAD"
arg_image_number=50

file_list = glob.glob("./test/*/*")
arg_cpu_extension=None

arg_labels="./mn-labels.txt"
label_file = open(arg_labels, "r")
labels = label_file.read().split('\n')

def async_infer_worker(exec_net, image_queue, input_blob, out_blob):
    global start_time

    current_inference, next_inference = 0, 1
    inferred_images = 0
    while True:
        image = image_queue.get()
        if type(image) != np.ndarray:
                break
        exec_net.start_async(request_id=current_inference, inputs={input_blob: image})
        if exec_net.requests[next_inference].wait(-1) == 0:
            inferred_images = inferred_images + 1
            res = exec_net.requests[next_inference].outputs[out_blob]
            top = res[0].argsort()[-1:][::-1]
            pred_label = labels[top[0]]
            print("predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])
        current_inference, next_inference = next_inference, current_inference
    # one more inference result left to check
    if exec_net.requests[next_inference].wait(-1) == 0:
        inferred_images = inferred_images + 1
        res = exec_net.requests[next_inference].outputs[out_blob]
        top = res[0].argsort()[-1:][::-1]
        pred_label = labels[top[0]]
        print("predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])
        
    duration = time.time() - start_time
    print("inferred frames:", inferred_images ,"average fps:",inferred_images/duration)
    
def image_process_worker(image_queue, n, c, h, w):
    for i in range(1, 1 + arg_image_number):
        img_path = random.choice(file_list)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (w, h))
        image = image.transpose((2, 0, 1))
        image = image.reshape((n, c, h, w))
        image = preprocess_input(image)
        image_queue.put(image)
    image_queue.put(None)

start_time = -1
         
def main():
    global start_time

    image_queue = queue.Queue(maxsize= 4)
    
    plugin = IEPlugin(device=arg_device)
    net = IENetwork(model=arg_model_xml, weights=arg_model_bin)
    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
    n, c, h, w = net.inputs[input_blob].shape
    exec_net = plugin.load(network=net, num_requests=2)

    start_time = time.time()

    preprocess_thread = None
    preprocess_thread = threading.Thread(target=image_process_worker, args=(image_queue, n, c, h, w))
    preprocess_thread.start()
    
    async_infer_worker(exec_net, image_queue, input_blob, out_blob)
    
    preprocess_thread.join()
    
    del exec_net
    del net
    del plugin


if __name__ == '__main__':
    main()

## Lab4: inference sample code using async API, with multi-thread and multi-requests support  (for iv3/mn/vgg16 model)
### *arg_model_xml* : the model xml (generated from mo.py)
### *arg_model_bin* : the model bin (generated from mo.py)
### *arg_device* : the device to run inferences, can be one of CPU/GPU/MYRIAD
### *arg_image_number* : total infer requests to run
### *arg_request_number* : infer requests that run simultaneously

In [ ]:
#!/usr/bin/env python3
# async api + multi_threads + multi-requests
from openvino.inference_engine import IENetwork, IEPlugin
from keras.applications.mobilenet import preprocess_input
import cv2
import os
import sys
import random
import time
import glob

import numpy as np
from openvino.inference_engine import IENetwork, IEPlugin

from multiprocessing import Process, Queue
import multiprocessing
import threading
import queue

# args
arg_model_xml="./tf_model/mn/FP16/all_layers.mn.xml"
arg_model_bin="./tf_model/mn/FP16/all_layers.mn.bin"
arg_device="MYRIAD"
arg_image_number = 50
arg_request_number = 4

file_list = glob.glob("./test/*/*")
arg_cpu_extension=None

arg_labels="./mn-labels.txt"
label_file = open(arg_labels, "r")
labels = label_file.read().split('\n')

def async_infer_worker(exe_net, request_number, image_queue, input_blob, out_blob):
    global start_time
    
    current_request_ids = range(request_number)
    next_request_ids = range(request_number, request_number * 2)
    done = False
    last_batch = -1
    inferred_images = 0
    while True:
        buffers = []
        for i in range(request_number):
            b = image_queue.get()
            if type(b) != np.ndarray:
                buffers.append(None)
                done = True
                break
            else:
                buffers.append(b)
        for _request_id in current_request_ids:
            if _request_id >=  request_number:
                if type(buffers[_request_id - request_number]) == np.ndarray:
                    exe_net.start_async(request_id=_request_id, inputs={input_blob: buffers[_request_id - request_number]})
                else:
                    #print("image at index " + str(_request_id - request_number) + " is none." )
                    last_batch = _request_id - request_number
                    break
            else:
                if type(buffers[_request_id]) == np.ndarray:
                    exe_net.start_async(request_id=_request_id, inputs={input_blob: buffers[_request_id]})
                else:
                    #print("image at index " + str(_request_id) + " is none." )
                    last_batch = _request_id
                    break
                    
        for _request_id in next_request_ids:
            if exe_net.requests[_request_id].wait(-1) == 0:
                res = exe_net.requests[_request_id].outputs[out_blob]
                inferred_images = inferred_images + 1
                top = res[0].argsort()[-1:][::-1]
                pred_label = labels[top[0]]
                print("predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])

        current_request_ids, next_request_ids = next_request_ids, current_request_ids
        
        for i in range(len(buffers)):
            image_queue.task_done()
            
        if done:
            break

    # 'last_batch' more inference results remain to check
    buffer_index = 0
    for _request_id in next_request_ids:
        if(buffer_index >= last_batch):
            break
        buffer_index = buffer_index + 1
        if exe_net.requests[_request_id].wait(-1) == 0:
            res = exe_net.requests[_request_id].outputs[out_blob]
            inferred_images = inferred_images + 1
            top = res[0].argsort()[-1:][::-1]
            pred_label = labels[top[0]]
            print( "predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])
    duration = time.time() - start_time
    print("inferred frames:", inferred_images ,"average fps:",inferred_images/duration)

def preprocess_worker(image_queue, n, c, h, w):
    for i in range(1, 1 + arg_image_number):
        img_path = random.choice(file_list)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (w, h))
        image = image.transpose((2, 0, 1))
        image = image.reshape((n, c, h, w))
        image = preprocess_input(image)
        image_queue.put(image)
    image_queue.put(None)

start_time = -1
           
def main():
    global start_time
    
    image_queue = queue.Queue(maxsize= arg_request_number*3)

    plugin = IEPlugin(device=arg_device)
    net = IENetwork(model=arg_model_xml, weights=arg_model_bin)
    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
    n, c, h, w = net.inputs[input_blob].shape
    exec_net = plugin.load(network=net, num_requests=arg_request_number*2)

    start_time = time.time()

    preprocess_thread = None
    preprocess_thread = threading.Thread(target=preprocess_worker, args=(image_queue, n, c, h, w))
    preprocess_thread.start()
    
    async_infer_worker(exec_net, arg_request_number, image_queue, input_blob, out_blob)

    preprocess_thread.join()
    
    del exec_net
    del net
    del plugin


if __name__ == '__main__':
    main()

## Lab5: inference sample code using async API, with multi-thread, multi-requests and multi-ncs support  (for iv3/mn/vgg16 model)
### *arg_model_xml* : the model xml (generated from mo.py)
### *arg_model_bin* : the model bin (generated from mo.py)
### *arg_device* : the device to run inferences, can be one of CPU/GPU/MYRIAD
### *arg_image_number* : total infer requests to run
### *arg_request_number* : infer requests that run simultaneously
### *arg_ncs_number* :  ncs number

In [ ]:
#!/usr/bin/env python3
# async api + multi-threads + multi-requests + multi-ncs
from openvino.inference_engine import IENetwork, IEPlugin
from keras.applications.mobilenet import preprocess_input
import cv2
import os
import sys
import random
import time
import glob

import numpy as np
from openvino.inference_engine import IENetwork, IEPlugin

from multiprocessing import Process, Queue
import multiprocessing
import threading
import queue

# args
arg_model_xml="./tf_model/mn/FP16/all_layers.mn.xml"
arg_model_bin="./tf_model/mn/FP16/all_layers.mn.bin"
arg_device="MYRIAD"
arg_image_number = 100
arg_request_number = 4
arg_ncs_number = 2

file_list = glob.glob("./test/*/*")
arg_cpu_extension=None
arg_labels="./mn-labels.txt"
label_file = open(arg_labels, "r")
labels = label_file.read().split('\n')


inferred_images = 0
def async_infer_worker(exe_net, request_number, image_queue, input_blob, out_blob):
    global start_time
    global inferred_images
    global arg_image_number
    
    current_request_ids = range(request_number)
    next_request_ids = range(request_number, request_number * 2)
    done = False
    last_batch = -1
    while True:
        buffers = []
        for i in range(request_number):
            b = image_queue.get()
            if type(b) != np.ndarray:
                buffers.append(None)
                done = True
                break
            else:
                buffers.append(b)
        for _request_id in current_request_ids:
            if _request_id >=  request_number:
                if type(buffers[_request_id - request_number]) == np.ndarray:
                    exe_net.start_async(request_id=_request_id, inputs={input_blob: buffers[_request_id - request_number]})
                else:
                    #print("image at index " + str(_request_id - request_number) + " is none." )
                    last_batch = _request_id - request_number
                    break
            else:
                if type(buffers[_request_id]) == np.ndarray:
                    exe_net.start_async(request_id=_request_id, inputs={input_blob: buffers[_request_id]})
                else:
                    #print("image at index " + str(_request_id) + " is none." )
                    last_batch = _request_id
                    break
                    
        for _request_id in next_request_ids:
            if exe_net.requests[_request_id].wait(-1) == 0:
                res = exe_net.requests[_request_id].outputs[out_blob]
                inferred_images = inferred_images + 1
                top = res[0].argsort()[-1:][::-1]
                pred_label = labels[top[0]]
                print("predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])

        current_request_ids, next_request_ids = next_request_ids, current_request_ids
        
        for i in range(len(buffers)):
            image_queue.task_done()
            
        if done:
            break

    # 'last_batch' more inference results remain to check
    buffer_index = 0
    for _request_id in next_request_ids:
        if(buffer_index >= last_batch):
            break
        buffer_index = buffer_index + 1
        if exe_net.requests[_request_id].wait(-1) == 0:
            res = exe_net.requests[_request_id].outputs[out_blob]
            inferred_images = inferred_images + 1
            top = res[0].argsort()[-1:][::-1]
            pred_label = labels[top[0]]
            print("predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])
    if inferred_images == arg_image_number:
        duration = time.time() - start_time
        print("inferred frames:", inferred_images ,"average fps:",inferred_images/duration)

def preprocess_worker(image_queue, ncs_number, n, c, h, w):
    global arg_image_number
    
    for i in range(1, 1 + arg_image_number):
        img_path = random.choice(file_list)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (w, h))
        image = image.transpose((2, 0, 1))
        image = image.reshape((n, c, h, w))
        image = preprocess_input(image)
        image_queue.put(image)
    for i in range(ncs_number):
        image_queue.put(None)

start_time = -1

def main():
    global start_time
    global arg_ncs_number
    
    image_queue = queue.Queue(maxsize= arg_ncs_number*arg_request_number*4)

    plugin = IEPlugin(device=arg_device)
    net = IENetwork(model=arg_model_xml, weights=arg_model_bin)
    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
    n, c, h, w = net.inputs[input_blob].shape
    
    exec_nets = []
    for i in range(arg_ncs_number):
        exec_net = plugin.load(network=net, num_requests=arg_request_number*2)
        exec_nets.append(exec_net)

    start_time = time.time()

    preprocess_thread = threading.Thread(target=preprocess_worker, args=(image_queue, arg_ncs_number, n, c, h, w), daemon=True)
    preprocess_thread.start()
    
    infer_threads = [] 
    for f in range(arg_ncs_number):
        _worker = threading.Thread(target=async_infer_worker, args=(exec_nets[f], arg_request_number, image_queue, input_blob, out_blob))
        _worker.start()
        infer_threads.append(_worker)

    preprocess_thread.join()
    for _inf_thread in infer_threads:
        _inf_thread.join()
    
    for exec_net in exec_nets:
        del exec_net
    del net
    del plugin


if __name__ == '__main__':
    main()

***
## Optional - Lab6: inference sample code using async API, with multi-process, multi-requests and multi-ncs support  (for iv3/mn/vgg16 model)
### *arg_model_xml* : the model xml (generated from mo.py)
### *arg_model_bin* : the model bin (generated from mo.py)
### *arg_device* : the device to run inferences, can be one of CPU/GPU/MYRIAD
### *arg_image_number* : total infer requests to run
### *arg_request_number* : infer requests that run simultaneously
### *arg_ncs_number* : ncs number

In [ ]:
#!/usr/bin/env python3
# async api + multi-processes + multi-requests + multi-ncs
from openvino.inference_engine import IENetwork, IEPlugin
from keras.applications.mobilenet import preprocess_input
import cv2
import os
import sys
import time
import random
import glob

import numpy as np
from openvino.inference_engine import IENetwork, IEPlugin

from multiprocessing import Process, Queue
import multiprocessing
import threading
import queue

# args
arg_model_xml="./tf_model/mn/FP16/all_layers.mn.xml"
arg_model_bin="./tf_model/mn/FP16/all_layers.mn.bin"
arg_device="MYRIAD"
arg_labels="./mn-labels.txt"
arg_image_number = 100
arg_request_number = 4
arg_ncs_number = 2

file_list = glob.glob("./test/*/*")
arg_cpu_extension=None
label_file = open(arg_labels, "r")
labels = label_file.read().split('\n')


inferred_images = 0
def async_infer_worker(exe_net, request_number, image_queue, input_blob, out_blob):
    global start_time
    global inferred_images
    global arg_image_number
    
    current_request_ids = range(request_number)
    next_request_ids = range(request_number, request_number * 2)
    done = False
    last_batch = -1
    while True:
        buffers = []
        for i in range(request_number):
            b = image_queue.get()
            if type(b) != np.ndarray:
                buffers.append(None)
                done = True
                break
            else:
                buffers.append(b)
        for _request_id in current_request_ids:
            if _request_id >=  request_number:
                if type(buffers[_request_id - request_number]) == np.ndarray:
                    exe_net.start_async(request_id=_request_id, inputs={input_blob: buffers[_request_id - request_number]})
                else:
                    #print("image at index " + str(_request_id - request_number) + " is none." )
                    last_batch = _request_id - request_number
                    break
            else:
                if type(buffers[_request_id]) == np.ndarray:
                    exe_net.start_async(request_id=_request_id, inputs={input_blob: buffers[_request_id]})
                else:
                    #print("image at index " + str(_request_id) + " is none." )
                    last_batch = _request_id
                    break
                    
        for _request_id in next_request_ids:
            if exe_net.requests[_request_id].wait(-1) == 0:
                res = exe_net.requests[_request_id].outputs[out_blob]
                inferred_images = inferred_images + 1
                top = res[0].argsort()[-1:][::-1]
                pred_label = labels[top[0]]
                print("predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])

        current_request_ids, next_request_ids = next_request_ids, current_request_ids
        
        #for i in range(len(buffers)):
        #    image_queue.task_done()
            
        if done:
            break

    # 'last_batch' more inference results remain to check
    buffer_index = 0
    for _request_id in next_request_ids:
        if(buffer_index >= last_batch):
            break
        buffer_index = buffer_index + 1
        if exe_net.requests[_request_id].wait(-1) == 0:
            res = exe_net.requests[_request_id].outputs[out_blob]
            inferred_images = inferred_images + 1
            top = res[0].argsort()[-1:][::-1]
            pred_label = labels[top[0]]
            print("predict result:", '{:25s}'.format(pred_label), "confidence:", res[0][top[0]])
    if inferred_images == arg_image_number:
        duration = time.time() - start_time
        print("inferred frames:", inferred_images ,"average fps:",inferred_images/duration)

def preprocess_worker(image_queue, ncs_number, n, c, h, w):
    
    for i in range(1, 1 + arg_image_number):
        img_path = random.choice(file_list)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (w, h))
        image = image.transpose((2, 0, 1))
        image = image.reshape((n, c, h, w))
        image = preprocess_input(image)
        image_queue.put(image)
    for i in range(ncs_number):
        image_queue.put(None)

start_time = -1

def main():
    global start_time
    global arg_ncs_number
    
    image_queue = multiprocessing.Queue(maxsize= arg_ncs_number*arg_request_number*4)

    plugin = IEPlugin(device=arg_device)
    net = IENetwork(model=arg_model_xml, weights=arg_model_bin)
    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
    n, c, h, w = net.inputs[input_blob].shape
    
    exec_nets = []
    for i in range(arg_ncs_number):
        exec_net = plugin.load(network=net, num_requests=arg_request_number*2)
        exec_nets.append(exec_net)

    start_time = time.time()

    preprocess_process = multiprocessing.Process(target=preprocess_worker, args=(image_queue, arg_ncs_number, n, c, h, w), daemon=True)
    preprocess_process.start()
    
    infer_threads = [] 
    for f in range(arg_ncs_number):
        _worker = threading.Thread(target=async_infer_worker, args=(exec_nets[f], arg_request_number, image_queue, input_blob, out_blob))
        _worker.start()
        infer_threads.append(_worker)

    preprocess_process.join()
    for _inf_thread in infer_threads:
        _inf_thread.join()
    
    for exec_net in exec_nets:
        del exec_net
    del net
    del plugin

if __name__ == '__main__':
    main()